In [1]:
import sourcecode.Text.generate
import $ivy.`org.apache.spark::spark-sql:3.5.2`
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SQLContext
import java.nio.file.attribute.BasicFileAttributes
import java.nio.file._
import scala.collection.mutable.ArrayBuffer

Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.14.0-RC8/almond-spark_2.12-0.14.0-RC8.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/3.5.2/spark-sql_2.12-3.5.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/3.5.2/spark-parent_2.12-3.5.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-core/1.9.4/orc-core-1.9.4.pom
Downloaded https://repo1.maven.org/maven2/org/rocksdb/rocksdbjni/8.3.2/rocksdbjni-8.3.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-column/1.13.1/parquet-column-1.13.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-mapreduce/1.9.4/orc-mapreduce-1.9.4.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.11/scala-library-2.12.11.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-tags_2.12/3.5.2/spark-tags_2.1

import sourcecode.Text.generate
import $ivy.$
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SQLContext
import java.nio.file.attribute.BasicFileAttributes
import java.nio.file._
import scala.collection.mutable.ArrayBuffer

In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import org.apache.spark.SparkConf
val spark = SparkSession.builder.master("spark://spark:7077").config("spark.executor.memory", "10g").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
val sqlContext = spark.sqlContext
import sqlContext.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/12/19 12:48:03 INFO SparkContext: Running Spark version 3.5.2
24/12/19 12:48:03 INFO SparkContext: OS info Linux, 6.8.0-50-generic, amd64
24/12/19 12:48:03 INFO SparkContext: Java version 1.8.0_402
24/12/19 12:48:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/19 12:48:03 INFO ResourceUtils: ==============================================================
24/12/19 12:48:03 INFO ResourceUtils: No custom resources configured for spark.driver.
24/12/19 12:48:03 INFO ResourceUtils: ==============================================================
24/12/19 12:48:03 INFO SparkContext: Submitted application: 0ab15822-e2e9-43d8-9fe1-333ed4271ff6
24/12/19 12:48:03 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(memory -> name: memory, amount: 10240, script: , vendor: , offHeap -> name: offHeap, amount:

import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import org.apache.spark.SparkConf
spark: SparkSession = org.apache.spark.sql.SparkSession@1acf0570
sqlContext: SQLContext = org.apache.spark.sql.SQLContext@5b9bd4e2
import sqlContext.implicits._

In [3]:
def search_files(path: String, extension: String): ArrayBuffer[Path] = {
  val files = ArrayBuffer.empty[Path]
  val root = Paths.get(path)

  Files.walkFileTree(root, new SimpleFileVisitor[Path] {
    override def visitFile(file: Path, attrs: BasicFileAttributes) = {
      if (file.getFileName.toString.endsWith(extension)) {
        files += file
      }
      FileVisitResult.CONTINUE
    }
  })

  files
}

defined function search_files

In [ ]:
// Load parquet files into a DataFrame array
val df_array = search_files("/main/external/data/source_as_parquet", ".parquet")
  .map { f =>
    spark.read
      .format("parquet")
      .load(f.toString)
  }
// Concat all
val df = df_array.reduce((df1, df2) => df1.unionByName(df2, allowMissingColumns=true))
df.createOrReplaceTempView("df")
df.show(5)

java.nio.file.NoSuchFileException: /main/data/source_as_parquet

In [ ]:
val windowSpec = Window.partitionBy("file_name").orderBy("time")
var dv = df.select(
      $"Time".as("time"),
      $"Position".as("position"),
      $"Velocity".as("original_velocity"),
      $"Pump".as("pump"),
      $"neural_data",
      $"file_name"
)
.withColumn("dt", 
  $"time" - when(
    (lag("time", 1).over(windowSpec)).isNull, 
    0
  ).otherwise(
    lag("time", 1).over(windowSpec)
  )
)
.withColumn("dx",
  $"position" - when(
    (lag("position", 1).over(windowSpec)).isNull, 
    0
  ).otherwise(
    lag("position", 1).over(windowSpec)
  )
)
.withColumn("velocity",
  when(
    $"dt" === 0, 
    0
  ).otherwise(
    $"dx" / $"dt"
  )
)
.withColumn("file_name", regexp_replace(col("file_name"), "\\.[^.]+$", ""))
.withColumn("pump", round($"pump"))
.na.drop()
.where("exists(neural_data, x -> x is not null)")
.withColumn("index", monotonically_increasing_id())
dv.createOrReplaceTempView("dv")
dv.show(5)

+------------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+-------------------+------------------+-----+
|              time|         position|original_velocity|pump|         neural_data|           file_name|                  dt|                 dx|          velocity|index|
+------------------+-----------------+-----------------+----+--------------------+--------------------+--------------------+-------------------+------------------+-----+
|0.9684967780795763|167.8834395309168|15.33076223119325| 0.0|[0.00292782419273...|ID17905/Day5/Data...|0.031241831550954058|0.11094945690160785|3.5513108993195277|    0|
|0.9997386096305304|168.0412270034912|13.94933848866475| 0.0|[0.00290607741044...|ID17905/Day5/Data...|0.031241831550954058|0.15778747257439818| 5.050519279481222|    1|
| 1.030980441181484|168.2425834116714|12.65062656640945| 0.0|[0.00314377625181...|ID17905/Day5/Data...|0.031241831550953614|0.20135640818020306| 6.445

windowSpec: WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3e8a4580
dv: org.apache.spark.sql.package.DataFrame = [time: double, position: double ... 8 more fields]

In [ ]:
//Positional encoding assuming that it starts on 0 and ends in maxPosition
val maxPosition = df.select(max($"position")).head()(0)
val pi = 3.141592653589793

dv = dv.withColumn("positional_encoding", sin($"position")*2.0*pi/maxPosition)

maxPosition: Any = 359.964959290941
pi: Double = 3.141592653589793

In [ ]:
dv.write.partitionBy("file_name").mode("overwrite").parquet("/main/workspace/transformed")